In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [32]:
import pandas as pd
import numpy as np

### Helper Functions

In [19]:
def rolling_zscore(data, window_size):
    # Calculate rolling mean and standard deviation
    rolling_mean = data.rolling(window_size).mean()
    rolling_std = data.rolling(window_size).std()

    # Calculate z-score
    zscore = (data - rolling_mean) / rolling_std

    return zscore

### Data Intake

In [8]:
tickers = list(pd.read_csv("ind_nifty200list.csv")["Symbol"])

In [11]:
data = pd.read_csv("data.csv").set_index("Datetime")

MOM_ac: Price Momentum (12 months skip current month)

In [20]:
for ticker in tickers:
    data[f"{ticker}_MOM_ac"] = rolling_zscore(data[ticker].shift(30)/data[ticker].shift(365) - 1, 365)

MOM_vol: Price Momentum adjusted by daily vol (combination
of 13 month and 7 month price momentum adjusted
by daily return std deviation for the 13 month period)

In [26]:
for ticker in tickers:
    data[f"{ticker}_MOM_vol"] = 0.5*rolling_zscore(data[ticker].shift(30)/data[ticker].shift(13*30) - 1, 365)/data[ticker].rolling(13*30).apply(lambda x: x[:-30].std())+\
                                0.5*rolling_zscore(data[ticker].shift(30)/data[ticker].shift(7*30) - 1, 365)/data[ticker].rolling(13*30).apply(lambda x: x[:-30].std())

### LOVOL: 12 month daily price log return volatility from M-1 to M-13

In [39]:
for ticker in tickers:
    data[f"{ticker}_LOVOL"] = rolling_zscore(1/np.log(data[ticker]/data[ticker].shift(365)).rolling(13*30).apply(lambda x: x[:-30].std()),20)